# Summary Chapter 6 - Key point model

In this example we are going to train a model to identify a certain point in an image. In this case it is the center of the head in images of people. The dataset we are using is called Biwi Kinect Head Pose dataset. 

Let's first include everthing we need and fetch the data:

In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *

In [ ]:
path = untar_data(URLs.BIWI_HEAD_POSE)
Path.BASE_PATH = path

The data comprises 24 folders (for 24 different people). Each of those comprises a number of images (poses) with each a txt file which specifies the coordinates of the head center key point.<br>
Let's get the image files and name them according to the poses.

In [ ]:
img_files = get_image_files(path)
def img2pose(x): return Path(f'{str(x)[:-7]}pose.txt')
img2pose(img_files[0])

We can take a look at shape and a thump of one image using:<br>
```python
im = PILImage.create(img_files[0])
im.shape<br>
```
and<br>
```python
im.to_thumb(160)
```

We need to define a fct that turns the txt file into a two item tensor with the coordinates of the key points. In the case of the Biwi data this is:

In [ ]:
cal = np.genfromtxt(path/'01'/'rgb.cal', skip_footer=6)
def get_ctr(f):
    ctr = np.genfromtxt(img2pose(f), skip_header=3)
    c1 = ctr[0] * cal[0][0]/ctr[2] + cal[0][2]
    c2 = ctr[1] * cal[1][1]/ctr[2] + cal[1][2]
    return tensor([c1,c2])

With this we can create a DataBlock object with an ImageBlock and a PointBlock. With the present data it is important to not randomly split all files, since it's only 24 people with different poses. We will use only one person's images as validation set. Hence, we make sure that the training hasn't seen this person before. <br>
When using a Pointblock fastAI knows that we are dealing with coordinates and applies any augmentation also to the coordinates.

In [ ]:
biwi = DataBlock(
    blocks=(ImageBlock, PointBlock),
    get_items=get_image_files,
    get_y=get_ctr,
    splitter=FuncSplitter(lambda o: o.parent.name=='13'),
    batch_tfms=[*aug_transforms(size=(240,320)), 
                Normalize.from_stats(*imagenet_stats)]
)

Now for the DataLoader. We also want to check a minibatch: 

In [ ]:
dls = biwi.dataloaders(path)
dls.show_batch(max_n=9, figsize=(8,6))

Next we create the learner. We are also going to specify the range of the targets (coordinates). Coordinates always get rescaled to -1:1, hence the y_range.

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(-1,1))

If the loss fct isn't explicitly defined fastAI is going to choose automatically. In this case

```python
dls.loss_func
```
is FlattenedLoss of MSELoss()

We can use the lr-finder to find a good learn rate, use it to fine tune and take a look at the results:

In [ ]:
learn.lr_find()

In [ ]:
lr = 1e-2
learn.fine_tune(3, lr)

In [ ]:
learn.show_results(ds_idx=1, nrows=3, figsize=(6,8))